# Dans ce Notebook, les inputs, donc l'anglais va etre en sparse encoding, tandis que le francais sera en one hot encoding

le one hot encoding c'est lorsque les lettres sont remplacées par des matrice de taille = au nombre de caractere.
Ces matrices sont pleines de  sauf a l'indice du caractere codé, ou il y a un 

le sparse, c'est juste que chaque lettre est remplacé directement par un nombre

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
from string import punctuation

total_sentence = 50000

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
df_translation = pd.read_csv('/content/drive/MyDrive/DATASET/machine_translation/eng_-french.csv',nrows=total_sentence)
print('longueur dataset',len(df_translation))

longueur dataset 50000


In [35]:
df_translation.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


# Cleaning text

In [36]:

to_delete_en = {'!','1','2','3','4','5','6','7','8','9','0','ú', ',', '’', '\xa0', 'º', '.', '?', '-', '+', '‘', "'", 'ç', '\xad',
                ':', '—','–', '&', '/', 'ö', ';', '₂', '€', '%', '$', '"'} 


#for sentence in df_translation['English words/sentences']:

def clean_en(sentence):
  sentence = sentence.lower()
  for char in sentence:
    if char in to_delete_en:
      sentence = sentence.replace(char, ' ')
  return sentence

df_translation["English words/sentences"] = df_translation["English words/sentences"].apply(lambda x: clean_en(x))



In [37]:

to_delete_fr = {'!', '5',  ',',  'Ç', '’', '\xa0', ')', '…',  '6', '1', ' ', 'É', 'À',
                '.', '7', '?', '-', 'Â', '+', '2',
                '‘', "'", '‽', 'î', '«', ':'
                , '3', '8', '–', '\u202f', 'Ê', '&', '/', '0', 'Ô',
                 '(', '\u2009', '»', '₂', ';', '9', '4',  '%', '\u200b', '$', '"'}
def clean_fr(sentence):
  sentence = sentence.lower()
  for char in sentence:
    if char in to_delete_fr:
      sentence = sentence.replace(char, ' ')
  return sentence
df_translation["French words/sentences"] = df_translation["French words/sentences"].apply(lambda x: clean_fr(x))


In [38]:
df_translation

,English words/sentences,French words/sentences
0,hi,salut
1,run,cours
2,run,courez
3,who,qui
4,wow,ça alors
...,...,...
49995,tell us all the gossip,racontez nous tous les commérages
49996,ten years have gone by,dix ans se sont écoulés
49997,tennis is loads of fun,le tennis est extrêmement divertissant
49998,thank you all for that,merci à tous pour cela


# Calcul de la longueur max des sequences 

In [39]:
max_len_en = max([len(x) for x in df_translation['English words/sentences']])
max_len_fr = max([len(x) for x in df_translation['French words/sentences']])
print('longueur max sequence fr ',max_len_fr)
print('longueur max sequence en ',max_len_en)

longueur max sequence fr  72
longueur max sequence en  23


# Creation du vocabulaire

In [40]:
char_fr = {c for word in df_translation['French words/sentences'] for c in word}
print(char_fr)
print('longueur vocab fr',len(char_fr))

{'m', 'é', 't', 'o', 's', 'è', 'r', 'z', 'v', 'b', 'k', 'w', 'ï', 'a', 'x', 'u', 'â', 'j', 'e', 'ê', 'ë', 'ç', 'œ', 'ù', 'c', 'f', 'p', 'h', 'l', 'à', 'g', 'q', 'i', 'd', 'y', 'û', ' ', 'n', 'ô'}
longueur vocab fr 39


In [41]:
char_en = {c for word in df_translation['English words/sentences'] for c in word}
print(char_en)
print('longueur vocab en',len(char_en))

{'m', 'é', 't', 'o', 's', 'r', 'z', 'v', 'b', 'k', 'w', 'a', 'x', 'u', 'j', 'e', 'f', 'h', 'p', 'c', 'l', 'g', 'i', 'q', 'd', 'y', ' ', 'n'}
longueur vocab en 28


# Association char to ind 

In [42]:
char_to_ind_en = {char : ind for ind,char in enumerate(char_en)}
ind_to_char_en = {ind : char for ind,char in enumerate(char_en)}

char_to_ind_fr = {char : ind for ind,char in enumerate(char_fr)}
ind_to_char_fr = {ind : char for ind,char in enumerate(char_fr)}

In [43]:
print(df_translation['English words/sentences'][0])

hi 


# Conversion en one hot encoding de l'output ( francais )

# Dimension :  ( nombre fichiers : 50000 , nombre max de caractere : 72, nombre de caracteres differents : 42)

# Initialisation des matrices de One Hot encoding

In [44]:
encoded_fr = np.zeros((len(df_translation),72,40))

# Conversion du sbt et bdd en one hot encoding

In [45]:
from tqdm import tqdm

In [46]:
for i,sentence in tqdm( enumerate(df_translation['French words/sentences'])):
    for j,char in enumerate (sentence):
        encoded_fr[i,j,char_to_ind_fr[char]] = 1.

50000it [00:00, 106519.19it/s]


In [47]:
encoded_fr.shape

(50000, 72, 40)

In [48]:
trainY = encoded_fr

# Decodage du one hot encoding

In [49]:
encoded_sequence_fr = df_translation['French words/sentences'][0]
encoded_sequence_fr

'salut '

In [50]:
def decode_fr(encoded_one_hot_fr):
  decoded_sequence_fr = []
  for i,chars in enumerate(encoded_one_hot_fr):
    for j,char in enumerate(chars):
      if (char==1):
        decoded_sequence_fr.append(ind_to_char_fr[j])

  decoded_sequence_fr = ''.join(decoded_sequence_fr)
  return decoded_sequence_fr

In [51]:
decoded_fr = decode_fr(trainY[0])
print(decoded_fr)

salut 


# Encodage de l'anglais

# Encodage en char to char but sparse

# dimmension : ( nombre fichiers : 50000, nombre de char max : 23)

In [52]:
ind_space_en = char_to_ind_en[' ']

In [53]:
encoded_en = np.full((len(df_translation),23),ind_space_en)

In [54]:
for i,chars in enumerate(df_translation['English words/sentences']):
    for j,char in enumerate (chars):
        encoded_en[i,j] = char_to_ind_en[char]

In [55]:
print(encoded_en.shape)

(50000, 23)


In [56]:
trainX = encoded_en

# Decodage en sparse char to char

In [57]:
def decode_en(encoded):
  decoded_en = []
  for char in encoded:
    decoded_en.append(ind_to_char_en[char])
  decoded_en = ''.join(decoded_en)
  return decoded_en


In [58]:
decoded_en = decode_en(trainX[0])
print(decoded_en)

hi                     


# Creation du Modele

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, TimeDistributed, LSTM, Bidirectional, Embedding, RepeatVector
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [60]:
def create_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    # Create the model
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(Dense(tar_vocab, activation='softmax'))
    return model
 
# Create model
model = create_model(len(char_en)+1, len(char_fr)+1, 23, 72, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [61]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 23, 256)           7424      
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 72, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 72, 256)           525312    
_________________________________________________________________
dense_1 (Dense)              (None, 72, 40)            10280     
Total params: 1,068,328
Trainable params: 1,068,328
Non-trainable params: 0
_________________________________________________________________


In [62]:
model.fit(trainX, trainY,batch_size = 64, epochs=40, validation_split=0.1)

Epoch 1/40


KeyboardInterrupt: ignored

In [ ]:
losses = pd.DataFrame(model.history.history)
plt.plot(losses)
plt.show()

In [ ]:
model.save('v1_char_by_char_target_ohe.h5')

In [ ]:
#from tensorflow.keras.models import load_model
#model = load_model('/content/v1_char_by_char_target_ohe (2).h5')

In [ ]:
model.summary()

# Test

In [ ]:
def decode_prediction(prediction):
  sequence_prediction = []
  for i, char in enumerate(prediction):
    sequence_prediction.append(np.argmax(char))
  decoded_sequence = []
  for char in sequence_prediction:
    decoded_sequence.append(ind_to_char_fr[char])
  decoded_sequence = ''.join(decoded_sequence)
  return decoded_sequence

In [ ]:
for i in range ( 30 ):
  input  = trainX[i]

  print('sentence to translate :',decode_en(trainX[i]))
  print('true translation : ',decode_fr(trainY[i]))
  input = np.expand_dims(input,axis=0)
  prediction = model.predict(input)[0]

  print('predicted translation :',decode_prediction(prediction))
  print('\n')
